In [1]:
from conv import *
from fcn import *
from client import *

import numpy as np
import pandas as pd
import torch as torch
import torch.nn.functional as F
import crypten
torch.set_printoptions(precision=16)
torch.set_default_tensor_type(torch.DoubleTensor)

In [2]:
input_batch = 3
input_channel = 10
input_size = 128
filter_channel = 10
filter_size = 3
act = 'ReLU'
pooling_size = 2

In [3]:
conv_list = []
conv_list.append(Conv(input_batch, 3, input_size, filter_channel, filter_size, act, pooling_size))
input_size = (int)(input_size / 2)

while input_size > 1:
    conv_list.append(Conv(input_batch, input_channel, input_size, filter_channel, filter_size, act, pooling_size))
    input_size = (int)(input_size / 2)

In [4]:
fcn_list = []
output_size = 16

fcn1 = FCN(input_batch, filter_channel, output_size)
fcn_list.append(fcn1)

while output_size > 2:
    fcn_list.append(FCN(input_batch, output_size, (int)(output_size / 2)))
    output_size = (int)(output_size / 2)

In [5]:
def train(data):
    hidden_list = []
    hidden_list.append(conv_list[0].forward(data))
    
    for i in range(1, len(conv_list)):
        print(f'running convolution {i} ...')
        temp = conv_list[i].forward(hidden_list[-1])
        hidden_list.append(temp)
        
    hidden_list[-1] = hidden_list[-1].reshape(input_batch, 1, filter_channel)
        
    for fcn_layer in fcn_list:
        print(f'running full connection ...')
        temp = fcn_layer.forward(hidden_list[-1])
        hidden_list.append(temp)
        
    return hidden_list[-1]

In [6]:
client = Client(10)
data = client.get_data()

prediction = train(data)

loss = client.loss(prediction)

Reading /home/dhz/bci-data/ds003478-download/sub-010/eeg/sub-010_task-Rest_run-01_eeg.fdt


/home/dhz/anaconda3/lib/python3.9/site-packages/crypten/mpc/primitives/arithmetic.py:372: UserWarning: Defining your `__torch_function__` as a plain method is deprecated and will be an error in future, please define it as a classmethod. (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:295.)
  result.share = getattr(torch, op)(result.share, y, *args, **kwargs)
/home/dhz/anaconda3/lib/python3.9/site-packages/crypten/mpc/provider/tfp_provider.py:25: UserWarning: Defining your `__torch_function__` as a plain method is deprecated and will be an error in future, please define it as a classmethod. (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:295.)
  c = getattr(torch, op)(a, b, *args, **kwargs)
/home/dhz/anaconda3/lib/python3.9/site-packages/crypten/mpc/primitives/beaver.py:82: UserWarning: Defining your `__torch_function__` as a plain method is deprecated and will be an error in future, please define it as a classmethod. (Triggered internally at  ../

running convolution 1 ...
running convolution 2 ...
running convolution 3 ...
running convolution 4 ...
running convolution 5 ...
running convolution 6 ...
running full connection ...
running full connection ...
running full connection ...
running full connection ...


In [7]:
prediction.shape

torch.Size([3, 1, 2])

In [8]:
pred = prediction.get_plain_text()
pred.reshape(3, 2)
pred.shape

torch.Size([3, 1, 2])

In [9]:
loss

tensor(6.4804423692745441e+19, device='cuda:0', dtype=torch.float32)

In [10]:
learning_rate = 0.01
prev = client.loss_grad()

for i in range(len(fcn_list)):
    weight, prev = fcn_list[len(fcn_list) - 1 - i].backpropagation(prev, client, learning_rate)
    
prev = prev.reshape(input_batch, filter_channel, 1, 1)
    
for i in range(len(conv_list)):
    weight, prev = conv_list[len(conv_list) - 1 - i].backpropagation(prev, client, learning_rate)

/home/dhz/experiment-BCI/client.py:25: UserWarning: An output with one or more elements was resized since it had shape [2], which does not match the required output shape [3, 10, 2, 2]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  ../aten/src/ATen/native/Resize.cpp:17.)
  dp_dr = torch.eq(p_plain.repeat_interleave(2, dim=2).repeat_interleave(2, dim=3), relu_plain, out=torch.tensor([1.,0.]).cuda())
/home/dhz/experiment-BCI/client.py:28: UserWarning: An output with one or more elements was resized since it had shape [2], which does not match the required output shape [3, 10, 2, 2]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with 

In [11]:
client = Client(10)
data = client.get_data()

prediction = train(data)

loss = client.loss(prediction)

/home/dhz/anaconda3/lib/python3.9/site-packages/crypten/__init__.py:64: RuntimeWarning: CrypTen is already initialized.
  warnings.warn("CrypTen is already initialized.", RuntimeWarning)


Reading /home/dhz/bci-data/ds003478-download/sub-010/eeg/sub-010_task-Rest_run-01_eeg.fdt


/home/dhz/anaconda3/lib/python3.9/site-packages/crypten/mpc/primitives/arithmetic.py:372: UserWarning: Defining your `__torch_function__` as a plain method is deprecated and will be an error in future, please define it as a classmethod. (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:295.)
  result.share = getattr(torch, op)(result.share, y, *args, **kwargs)
/home/dhz/anaconda3/lib/python3.9/site-packages/crypten/mpc/provider/tfp_provider.py:25: UserWarning: Defining your `__torch_function__` as a plain method is deprecated and will be an error in future, please define it as a classmethod. (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:295.)
  c = getattr(torch, op)(a, b, *args, **kwargs)
/home/dhz/anaconda3/lib/python3.9/site-packages/crypten/mpc/primitives/beaver.py:82: UserWarning: Defining your `__torch_function__` as a plain method is deprecated and will be an error in future, please define it as a classmethod. (Triggered internally at  ../

running convolution 1 ...
running convolution 2 ...
running convolution 3 ...
running convolution 4 ...
running convolution 5 ...
running convolution 6 ...
running full connection ...
running full connection ...
running full connection ...
running full connection ...


In [12]:
loss

tensor(4.1915001731937206e+19, device='cuda:0', dtype=torch.float32)

In [13]:
client = Client(10)
data = client.get_data()

prediction = train(data)

loss = client.loss(prediction)

/home/dhz/anaconda3/lib/python3.9/site-packages/crypten/__init__.py:64: RuntimeWarning: CrypTen is already initialized.
  warnings.warn("CrypTen is already initialized.", RuntimeWarning)


Reading /home/dhz/bci-data/ds003478-download/sub-010/eeg/sub-010_task-Rest_run-01_eeg.fdt


/home/dhz/anaconda3/lib/python3.9/site-packages/crypten/mpc/primitives/arithmetic.py:372: UserWarning: Defining your `__torch_function__` as a plain method is deprecated and will be an error in future, please define it as a classmethod. (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:295.)
  result.share = getattr(torch, op)(result.share, y, *args, **kwargs)
/home/dhz/anaconda3/lib/python3.9/site-packages/crypten/mpc/provider/tfp_provider.py:25: UserWarning: Defining your `__torch_function__` as a plain method is deprecated and will be an error in future, please define it as a classmethod. (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:295.)
  c = getattr(torch, op)(a, b, *args, **kwargs)
/home/dhz/anaconda3/lib/python3.9/site-packages/crypten/mpc/primitives/beaver.py:82: UserWarning: Defining your `__torch_function__` as a plain method is deprecated and will be an error in future, please define it as a classmethod. (Triggered internally at  ../

running convolution 1 ...
running convolution 2 ...
running convolution 3 ...
running convolution 4 ...
running convolution 5 ...
running convolution 6 ...
running full connection ...
running full connection ...
running full connection ...
running full connection ...


In [14]:
loss

tensor(4.1915001731937206e+19, device='cuda:0', dtype=torch.float32)

In [15]:
pred = prediction.get_plain_text()
pred.reshape(3, 2)
pred.shape

learning_rate = 0.01
prev = client.loss_grad()

for i in range(len(fcn_list)):
    weight, prev = fcn_list[len(fcn_list) - 1 - i].backpropagation(prev, client, learning_rate)
    
prev = prev.reshape(input_batch, filter_channel, 1, 1)
    
for i in range(len(conv_list)):
    weight, prev = conv_list[len(conv_list) - 1 - i].backpropagation(prev, client, learning_rate)

/home/dhz/experiment-BCI/client.py:25: UserWarning: An output with one or more elements was resized since it had shape [2], which does not match the required output shape [3, 10, 2, 2]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  ../aten/src/ATen/native/Resize.cpp:17.)
  dp_dr = torch.eq(p_plain.repeat_interleave(2, dim=2).repeat_interleave(2, dim=3), relu_plain, out=torch.tensor([1.,0.]).cuda())
/home/dhz/experiment-BCI/client.py:28: UserWarning: An output with one or more elements was resized since it had shape [2], which does not match the required output shape [3, 10, 2, 2]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with 

In [16]:
client = Client(10)
data = client.get_data()

prediction = train(data)

loss = client.loss(prediction)

/home/dhz/anaconda3/lib/python3.9/site-packages/crypten/__init__.py:64: RuntimeWarning: CrypTen is already initialized.
  warnings.warn("CrypTen is already initialized.", RuntimeWarning)


Reading /home/dhz/bci-data/ds003478-download/sub-010/eeg/sub-010_task-Rest_run-01_eeg.fdt


/home/dhz/anaconda3/lib/python3.9/site-packages/crypten/mpc/primitives/arithmetic.py:372: UserWarning: Defining your `__torch_function__` as a plain method is deprecated and will be an error in future, please define it as a classmethod. (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:295.)
  result.share = getattr(torch, op)(result.share, y, *args, **kwargs)
/home/dhz/anaconda3/lib/python3.9/site-packages/crypten/mpc/provider/tfp_provider.py:25: UserWarning: Defining your `__torch_function__` as a plain method is deprecated and will be an error in future, please define it as a classmethod. (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:295.)
  c = getattr(torch, op)(a, b, *args, **kwargs)
/home/dhz/anaconda3/lib/python3.9/site-packages/crypten/mpc/primitives/beaver.py:82: UserWarning: Defining your `__torch_function__` as a plain method is deprecated and will be an error in future, please define it as a classmethod. (Triggered internally at  ../

running convolution 1 ...
running convolution 2 ...
running convolution 3 ...
running convolution 4 ...
running convolution 5 ...
running convolution 6 ...
running full connection ...
running full connection ...
running full connection ...
running full connection ...


In [17]:
loss

tensor(5.1401579260295512e+19, device='cuda:0', dtype=torch.float32)

In [18]:
prediction

MPCTensor(
	_tensor=CUDALongTensor(tensor([[[  58436303036957,  526929959574734]],

        [[ 237259700864000, -270505036971923]],

        [[-149419957694254,   93122587598527]]], device='cuda:0'))
	plain_text=HIDDEN
	ptype=ptype.arithmetic
)

In [19]:
prediction.get_plain_text()

tensor([[[ 8.9166720000000000e+08,  8.0403133440000000e+09]],

        [[ 3.6202956800000000e+09, -4.1276423680000000e+09]],

        [[-2.2800023040000000e+09,  1.4209379840000000e+09]]], device='cuda:0',
       dtype=torch.float32)